Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Training a GNN to do Fraud Detection on Graphcore IPUs using your own dataset with PyTorch Geometric

TODO: Everything in this section

TODO: Update links:

[![Run on Gradient](../../gradient-badge.svg)](https://console.paperspace.com/github/<runtime-repo>?machine=Free-IPU-POD4&container=<dockerhub-image>&file=<path-to-file-in-repo>)  [![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

>
> We aim to have our notebook app demos to be focused on what the user is trying to
> do. To help you do this correctly please read [our user personnas](https://graphcore.atlassian.net/wiki/spaces/PM/pages/3157131517/Notebook+personas#Ellie%3A-The-Data-Scientist%2C-Business-Analysis%2C-Consultant),
and when in doubt ask yourself "does that person care about this?".
> To support that the first paragraph will contain all the key information, to
> help users rapidly identify if this is the right notebook for them to go
> though, based on:
>
> - The task/business problem they are trying to solve,
> - The features that are used (Focus on big picture Deep learning features - e.g.
>  Distributed training, not I/O overlap).
>
> To achieve this, each notebook should start with the following 3 paragraphs
> (detailed in the next three comments):
>
> - a table highlighting what we are going to do
> - a very short intro (3-5 sentences)
> - clear "steps to resolution" (bullet points stating what the user will have
>    to do to tackle their problem on the IPU - these need to reflect the notebook,
>    and be as simple as possible)
> - links to additional related resources.
>

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
|   GNNs   |  Fraud detection  | ? | ? | Training, evaluation | recommended: 16XX (min: 4X) | 20Xmn (X1h20mn)   |

>
>
> Start with a short introduction to the notebook. [suggested 3-5 sentences]
>
> This intro should focus on the problem you are fixing, and not on any IPU specific
> or framework specific features. The mindset is that anything that is non-standard
> is a barrier to entry, and will risk the user giving up.
>
> This short introduction should be followed by a clear bullet point summary of
> the steps of the demo. Each outcome should be of the form:
> - what the user will do (active verb) [and (optionally) how they do
>   it]. Jargon, if any, goes to the end of the bullet point.

In this demo, you will learn how to:

- Turn tabular transaction data into a PyTorch Geometric dataset
- Select a model suitable for the task of predicting fraudulent transactions
- Train the model on Graphcore IPUs
- Run validation on the trained model

This notebook assumes some familiarity with PopTorch as well as PyTorch Geometric (PyG). For additional resources please consult:
* [PopTorch Documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/index.html)
* [PopTorch Examples and Tutorials](https://docs.graphcore.ai/en/latest/examples.html#pytorch)
* [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
* [PopTorch Geometric Documentation](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/en/latest/index.html)

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

In [1]:
# Make imported python modules automatically reload when the files are changed
# needs to be before the first import.
%load_ext autoreload
%autoreload 2
# TODO: remove at the end of notebook development

In [2]:
# TODO: Add gc-logger?

## Environment setup

[![Run on Gradient](../../gradient-badge.svg)](TODO)

The best way to try this demo is on Paperspace Gradient's cloud IPUs. To use on other hardware
make sure that you have the Poplar SDK enabled with the latest PopTorch Geometric installed.

In [3]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


To improve your experience we read some configuration related to the environment you are running the notebook in.

In [4]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))
pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod16")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")

# TODO Remove default
dataset_directory = os.getenv("DATASETS_DIR", ".")
checkpoint_directory = os.getenv("CHECKPOINT_DIR")

## Loading tabular data into PyTorch Geometric

### Getting the dataset

TODO: Using https://www.kaggle.com/c/ieee-fraud-detection/data

TODO: Run a script to download and tidy data?

In [5]:
from dataset import IeeeFraudDetectionDataset

dataset = IeeeFraudDetectionDataset(dataset_directory)
data = dataset[0]

In [65]:
data

tensor(1719)

### Visualize

In [66]:
#import networkx as nx
#from matplotlib import pyplot as plt
#from torch_geometric.utils import to_networkx
#
## Convert to homogeneous
#data_homogeneous = data.to_homogeneous()
#g = to_networkx(data_homogeneous)
## Use node types as colour map
#colour_map = data_homogeneous.node_type
#
## TODO: This maybe?
### Get labels
##labels = {str(idx): val for idx, val in enumerate(data_homogeneous.y)}
#
## Plot the graph
#nx.draw(g, node_color=colour_map, with_labels=True)
#plt.show()

## Preprocess

In [67]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)
data = T.NormalizeFeatures()(data)

data

HeteroData(
  transaction={
    num_nodes=1000,
    x=[1000, 963],
    y=[1000]
  },
  card1={
    num_nodes=415,
    x=[415, 1]
  },
  card2={
    num_nodes=132,
    x=[132, 1]
  },
  card3={
    num_nodes=16,
    x=[16, 1]
  },
  card4={
    num_nodes=4,
    x=[4, 1]
  },
  card5={
    num_nodes=35,
    x=[35, 1]
  },
  card6={
    num_nodes=2,
    x=[2, 1]
  },
  ProductCD={
    num_nodes=4,
    x=[4, 1]
  },
  addr1={
    num_nodes=51,
    x=[51, 1]
  },
  addr2={
    num_nodes=2,
    x=[2, 1]
  },
  P_emaildomain={
    num_nodes=32,
    x=[32, 1]
  },
  R_emaildomain={
    num_nodes=26,
    x=[26, 1]
  },
  (transaction, to, card1)={ edge_index=[2, 1000] },
  (transaction, to, card2)={ edge_index=[2, 992] },
  (transaction, to, card3)={ edge_index=[2, 1000] },
  (transaction, to, card4)={ edge_index=[2, 1000] },
  (transaction, to, card5)={ edge_index=[2, 995] },
  (transaction, to, card6)={ edge_index=[2, 1000] },
  (transaction, to, ProductCD)={ edge_index=[2, 1000] },
  (transa

### Create dataset splits

TODO: We sort by time above and then select those from more recent as the validation and test

In [68]:
num_nodes_train = int(0.8 * data["transaction"].num_nodes)
data["transaction"].train_mask = torch.zeros(data["transaction"].num_nodes, dtype=bool)
data["transaction"].train_mask[:num_nodes_train] = True
data["transaction"].val_mask = torch.zeros(data["transaction"].num_nodes, dtype=bool)
data["transaction"].val_mask[num_nodes_train:] = True

print(f"Number of training nodes: {data['transaction'].train_mask.sum()}")
print(f"Number of validation nodes: {data['transaction'].val_mask.sum()}")

Number of training nodes: 800
Number of validation nodes: 200


In [69]:
data

HeteroData(
  transaction={
    num_nodes=1000,
    x=[1000, 963],
    y=[1000],
    train_mask=[1000],
    val_mask=[1000]
  },
  card1={
    num_nodes=415,
    x=[415, 1]
  },
  card2={
    num_nodes=132,
    x=[132, 1]
  },
  card3={
    num_nodes=16,
    x=[16, 1]
  },
  card4={
    num_nodes=4,
    x=[4, 1]
  },
  card5={
    num_nodes=35,
    x=[35, 1]
  },
  card6={
    num_nodes=2,
    x=[2, 1]
  },
  ProductCD={
    num_nodes=4,
    x=[4, 1]
  },
  addr1={
    num_nodes=51,
    x=[51, 1]
  },
  addr2={
    num_nodes=2,
    x=[2, 1]
  },
  P_emaildomain={
    num_nodes=32,
    x=[32, 1]
  },
  R_emaildomain={
    num_nodes=26,
    x=[26, 1]
  },
  (transaction, to, card1)={ edge_index=[2, 1000] },
  (transaction, to, card2)={ edge_index=[2, 992] },
  (transaction, to, card3)={ edge_index=[2, 1000] },
  (transaction, to, card4)={ edge_index=[2, 1000] },
  (transaction, to, card5)={ edge_index=[2, 995] },
  (transaction, to, card6)={ edge_index=[2, 1000] },
  (transaction, to, Pr

In [70]:
num_fraud_train = data["transaction"].y[data["transaction"].train_mask].sum()
num_total_train = len(data["transaction"].train_mask)
num_fraud_val = data["transaction"].y[data["transaction"].val_mask].sum()
num_total_val = len(data["transaction"].val_mask)

In [71]:
# Number of fraud transactions
percentage_fraud_train = num_fraud_train / num_total_train
percentage_fraud_val = num_fraud_val / num_total_val
print(f"{percentage_fraud_train = :%}")
print(f"{percentage_fraud_val = :%}")

percentage_fraud_train = 3.800000%
percentage_fraud_val = 1.100000%


In [72]:
# Use this to set a class weight
class_weight = (
    (num_total_train / (2 * (num_total_train - num_fraud_train))).item(),
    (num_total_train / (2 * num_fraud_train)).item())
class_weight 

(0.5197505354881287, 13.1578950881958)

## Dataloading

TODO: Is graph too large that need to do some sampling - neighbour sampling

In [73]:
from torch_geometric.loader import NeighborLoader

batch_size = 128
num_layers = 2

train_loader = NeighborLoader(
    data,
    num_neighbors=[15] * num_layers,
    batch_size=batch_size,
    input_nodes=('transaction', data['transaction'].train_mask),
)

/nethome/adams/venvs/3.3.0+1361-EA.2/3.3.0+1361_poptorch/lib/python3.8/site-packages/torch_geometric/sampler/neighbor_sampler.py:58: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


In [74]:
from poptorch_geometric import FixedSizeOptions

# TODO: Use this
# fixed_size_options = FixedSizeOptions.from_loader(train_loader)
fixed_size_options = FixedSizeOptions(
    num_nodes=2000,
    num_edges=1000,
)

fixed_size_options

In [75]:
from draft_fixed_size_neighbour_loader import FixedSizeNeighborLoader

train_loader_ipu = FixedSizeNeighborLoader(
    data,
    num_neighbors=[15] * num_layers,
    fixed_size_options=fixed_size_options,
    batch_size=batch_size,
    input_nodes=('transaction', data['transaction'].train_mask),
    exclude_keys=("batch_size",),
)

In [76]:
sample = next(iter(train_loader_ipu))
sample

HeteroDataBatch(
  graphs_mask=[2],
  num_nodes=24000,
  num_edges=22000,
  transaction={
    num_nodes=2000,
    x=[2000, 963],
    y=[2000],
    train_mask=[2000],
    val_mask=[2000],
    n_id=[2000],
    input_id=[1326],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card1={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card2={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card3={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card4={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card5={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_mask=[2000]
  },
  card6={
    num_nodes=2000,
    x=[2000, 1],
    n_id=[2000],
    batch=[2000],
    ptr=[3],
    nodes_ma

## Picking the right model

TODO: Describe the task

TODO: Describe different relations type - want something that can do different relations (heterogeneous graph) - RGCN could be a sensible choice
    - but requires weights for every relation type - we have 11 relation types so might be ok

TODO: Try CompGCN
    - Only requires 3 weights - in, out, self loops

In [83]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import Linear, SAGEConv, to_hetero

import poptorch

# TODO: Include num layers?
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class Model(torch.nn.Module):

    def __init__(self,
                 hetero_gnn,
                 embedding_size,
                 out_channels,
                 class_weight=None,
                 batch_size=None):
        super().__init__()
        self.hetero_gnn = hetero_gnn
        self.embedding = nn.ModuleDict({
            node_type: nn.Embedding(data[node_type].num_nodes, embedding_size)
            for node_type in data.node_types
            if node_type != "transaction"
        })
        self.linear = Linear(-1, out_channels)
        self.full_batch = (batch_size is None)
        self.batch_size = batch_size
        self.class_weight = class_weight

    def forward(self,
                x_dict,
                edge_index_dict,
                n_id_dict=None,
                target=None,
                mask=None):
        for _, _, node_type in edge_index_dict.keys():
            if node_type != "transaction":
                if self.full_batch:
                    x_dict[node_type] = self.embedding[node_type].weight
                else:
                    assert n_id_dict is not None, (
                        "If using a sampled batch, `n_id_dict` must"
                        " be provided.")
                    x_dict[node_type] = self.embedding[node_type](n_id_dict[node_type])

        x_dict = self.hetero_gnn(x_dict, edge_index_dict)
        out = self.linear(x_dict['transaction'])
        if self.training:
            if self.full_batch:
                loss = F.cross_entropy(out, target, reduction='none')

                if self.class_weight is not None:
                    class_weight = target * self.class_weight[1] + (1 - target) * self.class_weight[0]
                    class_weight *= mask
                    class_weight *= (mask.sum() / class_weight.sum())
                    loss *= class_weight

                loss *= mask
                loss = loss.sum() / mask.sum()
                loss = poptorch.identity_loss(loss, reduction='none')
            else:
                out = out[:self.batch_size]
                target = target[:self.batch_size]
                # TODO: Use nodes_mask here

                loss = F.cross_entropy(out, target, reduction='none')

                if self.class_weight is not None:
                    class_weight = target * self.class_weight[1] + (1 - target) * self.class_weight[0]
                    class_weight *= (self.batch_size / class_weight.sum())
                    loss *= class_weight
                loss = poptorch.identity_loss(loss, reduction='mean')
            return out, loss
        return out


model = GNN(hidden_channels=64)
model = to_hetero(model, data.metadata(), aggr="sum")
model = Model(model,
              embedding_size=128,
              out_channels=2,
              class_weight=class_weight)

In [84]:
with torch.no_grad():  # Initialize lazy modules.
    out_cpu, loss = model(data.x_dict,
                          data.edge_index_dict,
                          target=data["transaction"].y,
                          mask=data["transaction"].train_mask)
out_cpu

tensor([[ 0.1387, -0.7291],
        [-0.9554, -0.1080],
        [-1.7077,  0.2353],
        ...,
        [ 0.9946, -0.0627],
        [-0.1329, -1.5922],
        [ 0.1098, -2.2154]])

In [85]:
import poptorch

model.eval()
inf_model = poptorch.inferenceModel(model)
out_ipu = inf_model(data.x_dict,
                    data.edge_index_dict,
                    target=data["transaction"].y,
                    mask=data["transaction"].train_mask)
out_ipu

[11:17:45.022] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[11:17:45.023] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
Graph compilation: 100%|██████████| 100/100 [01:09<00:00]


tensor([[ 0.1387, -0.7291],
        [-0.9554, -0.1080],
        [-1.7077,  0.2353],
        ...,
        [ 0.9946, -0.0627],
        [-0.1329, -1.5922],
        [ 0.1098, -2.2154]])

In [86]:
assert torch.allclose(out_cpu, out_ipu, rtol=1e-05, atol=1e-05)

## Training the model

TODO: Train the model in the normal way

In [87]:
learning_rate = 0.01
num_epochs = 20
embedding_size = 128
hidden_channels = 16
log_freq = 10

In [170]:
model = GNN(hidden_channels=hidden_channels)
model = to_hetero(model, data.metadata(), aggr="sum")
model = Model(model,
              embedding_size=embedding_size,
              #class_weight=class_weight,
              out_channels=2,
              batch_size=batch_size)

In [171]:
poptorch_options = poptorch.Options()
poptorch_options.enableExecutableCaching(executable_cache_dir)

Options(replication_factor=1, input_group_size=1, input_cgt=<CommGroupType.Consecutive: 1>, broadcast_buffers=True, device_iterations=1, log_dir='.', max_repeat_logs=4, auto_round_num_ipus=False, anchored_tensors={}, output_mode=4, output_return_period=1, connection_type=0, sync_pattern=0, available_memory_proportion={}, Distributed=_DistributedOptions(num_distributed_processes=1, distributed_process_id=0, ipuof_configs={}, _gcd_mappings={}, _warnings={}, _warnings_disabled=set(), _is_frozen=False), Jit=_JitOptions(, _warnings={}, _warnings_disabled=set(), _is_frozen=False), Precision=_PrecisionOptions(, _popart_options=_PopartOptions(instrumentWithHardwareCycleCounter=False, rearrangeAnchorsOnHost=False, cachePath='/tmp/exe_cache/', enableEngineCaching=True), _warnings={}, _warnings_disabled=set(), _is_frozen=False), TensorLocations=_TensorLocationOptions(, _warnings={}, _warnings_disabled=set(), _is_frozen=False), Training=_TrainingOptions(gradient_accumulation=1, accumulation_and_re

In [172]:
train_loader_ipu = FixedSizeNeighborLoader(
    data,
    num_neighbors=[15] * num_layers,
    fixed_size_options=fixed_size_options,
    batch_size=batch_size,
    input_nodes=('transaction', data['transaction'].train_mask),
    exclude_keys=("batch_size",),
    options=poptorch_options
)

In [173]:
sample = next(iter(train_loader_ipu))

with torch.no_grad():  # Initialize lazy modules.
    out_cpu, loss = model(sample.x_dict,
                          sample.edge_index_dict,
                          n_id_dict=sample.n_id_dict,
                          target=sample["transaction"].y)

loss

tensor(1.8612)

In [174]:
model.train()
optimizer = poptorch.optim.Adam(model.parameters(), lr=learning_rate)
training_model = poptorch.trainingModel(model, optimizer=optimizer, options=poptorch_options)

for epoch in range(num_epochs):
    total_examples = total_loss = 0
    for batch in train_loader_ipu:
        out, loss = training_model(batch.x_dict,
                                   batch.edge_index_dict,
                                   n_id_dict=batch.n_id_dict,
                                   target=batch['transaction'].y)
        total_examples += batch_size
        total_loss += float(loss) * batch_size

    if epoch % log_freq == 0:
        print(f"Epoch {epoch}, Loss: {total_loss / total_examples}")

[12:54:57.032] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[12:54:57.033] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[12:54:57.033] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.


Error: In poptorch/popart_compiler/source/CompilerImpl.cpp:725: 'poptorch_cpp_error': Failed to acquire 1 IPU(s)
Error raised in:
  [0] Compiler::initSession
  [1] LowerToPopart::compile
  [2] compileWithManualTracing


## Validating our trained model

TODO: Validate the model in the normal way

In [ ]:
# TODO: Should I sample?

model.eval()
model.full_batch = True
inference_model = poptorch.inferenceModel(model, options=poptorch_options)

out = inference_model(data.x_dict,
                      data.edge_index_dict,
                      target=data['transaction'].y,
                      mask=data['transaction'].val_mask)

[12:50:42.024] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
[12:50:42.026] [poptorch::python] [warning] Dicts as inputs only have partial support, they can be accessed using literal keys, but full Python functionality is not enabled. Consider changing dict inputs to tuple.
/nethome/adams/venvs/3.3.0+1361-EA.2/3.3.0+1361_poptorch/lib/python3.8/site-packages/torch/nn/modules/module.py:1802: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Graph compilation: 100%|██████████| 100/100 [00:05<00:00]


In [ ]:
y_pred = nn.Softmax(dim=-1)(out)
y_pred = y_pred[:, -1]
y_pred = y_pred > 0.5
y_pred = y_pred[data['transaction'].val_mask]
y_true = data['transaction'].y[data['transaction'].val_mask]

In [ ]:
def accuracy(y_pred, y_true):
    correct = y_pred.eq(y_true).sum()
    return correct / len(y_pred)

accuracy(y_pred, y_true)

tensor(0.8850)

In [ ]:
def get_confusion_matrix(y_pred, y_true):
    y_pred = y_pred.bool()
    y_true = y_true.bool()
    true_positives = (y_pred * y_true).sum()
    false_positives = (y_pred * ~y_true).sum()
    true_negatives = (~y_pred * ~y_true).sum()
    false_negatives = (~y_pred * y_true).sum()
    return true_positives, false_positives, true_negatives, false_negatives

true_pos, false_pos, true_neg, false_neg = get_confusion_matrix(y_pred, y_true)
true_pos, false_pos, true_neg, false_neg

(tensor(0), tensor(12), tensor(177), tensor(11))

In [ ]:
def get_rates(true_pos, false_pos, true_neg, false_neg):
    true_pos_rate = true_pos / (true_pos + false_neg)
    false_pos_rate = false_pos / (false_pos + true_neg)
    return true_pos_rate, false_pos_rate

get_rates(true_pos, false_pos, true_neg, false_neg)

(tensor(0.), tensor(0.0635))

In [ ]:
def precision(true_pos, false_pos):
    return true_pos / (true_pos + false_pos)

def recall(true_pos, false_neg):
    return true_pos / (true_pos + false_neg)

precision(true_pos, false_pos), recall(true_pos, false_neg)

(tensor(0.), tensor(0.))

### With threshold

In [ ]:
import numpy as np

results = []
for threshold in np.arange(0.0, 1.1, 0.1):
    y_pred = nn.Softmax(dim=-1)(out)
    y_pred = y_pred[:, -1]
    y_pred = y_pred > threshold
    y_pred = y_pred[data['transaction'].val_mask]
    y_true = data['transaction'].y[data['transaction'].val_mask]
    results.append((threshold, *get_rates(*get_confusion_matrix(y_pred, y_true))))

In [175]:
results

[(0.0, tensor(1.), tensor(1.)),
 (0.1, tensor(0.0909), tensor(0.0847)),
 (0.2, tensor(0.), tensor(0.0794)),
 (0.30000000000000004, tensor(0.), tensor(0.0794)),
 (0.4, tensor(0.), tensor(0.0741)),
 (0.5, tensor(0.), tensor(0.0635)),
 (0.6000000000000001, tensor(0.), tensor(0.0582)),
 (0.7000000000000001, tensor(0.), tensor(0.0582)),
 (0.8, tensor(0.), tensor(0.0529)),
 (0.9, tensor(0.), tensor(0.0476)),
 (1.0, tensor(0.), tensor(0.))]

## Explainability

TODO

## Conclusion

> The conclusion to your demo should:
>
> - summarise the main steps that were taken in the demo making clear what
>  your user got to do (similar to steps at the start but more
>  specific, you can link a specific feature/method/class to achieving a specific
>  outcome). (short paragraph: 3-6 sentences)
> - provide resources to go further: these can be links to other tutorials, to
>  documentation, to code examples in the public_examples repo, tech notes, deployments,
>  etc... (2-4 suggestions)
>
> For pointing users to notebooks in the same runtime, point the user to where the file is rather than a link. For example: please see our tutorial, `<folder_name>/<notebook_name>.ipynb`. For relative links the paperspace platform will download the file locally if the machine is running and if the machine is not running will 404. For full path links a new window is opened.